<h1><a href="https://arxiv.org/abs/1903.00589">
Formal Policy Learning from Demonstrations for Reachability Properties</a></h1>
by Hadi Ravanbakhsh et al.


<h2>Keynotes</h2>

* A counterexample-guided framework is designed to learn policy from demonstration while the learnt policy satisfies a formal specification. 

* Demonstration is generated by model-predictive controllers (MPCs). The underlying policy is solved via a nonlinear optimization problem and certified by a Lyapunov-like function.

* The policy is expressed as a linear combination of a set of basis functions and computed in real time.

<h1>Motivation</h1>

* Value function is harder than policy to learn.

* An adversarial component can actively identify and improve wrong policies.


<h1>Background</h1>

* Let $X\subset R^n$ be a state-space where vector $x\in X$ is a state. 
* Let $U\subset R^m$ be an action space where vector $u\in \mathcal{U}$ is a control input. 
* The plant model is $F: X\times \mathcal{U}\rightarrow R^n$. It satisfies the differential equation below.
$$\dot{x}=F(x,u), x\in X, u\in \mathcal{U}$$
* The policy $\pi:X\rightarrow \mathcal{U}$ is assumed to be deterministic and Lipshitz continuous over $X$. 
* A closed loop system $\Psi(F,\pi)$ with initial state $x_0\in X$ yields a trace $\sigma:R^+\rightarrow X$ that satisfies
$$\sigma(0)=x_0\ and\ (\forall t\geq 0)\ \dot{\sigma}(t)=F(\sigma(t), \pi(\sigma(t)))$$
* A specification $\varphi$ asserts $true/false$ for any given trace $\sigma$.
* `Reachability Property`: given a compact initial state set $I\subset X$ and a compact goal set $G\subset X$, the system must reach some state in $G:(\exists t\geq 0) \sigma(t)\in G$.
* `Policy Correctness`: for all traces $\sigma$ of $\Psi(F,\pi)$, $\varphi(\sigma)$ holds, $\Psi(F,\pi)\vDash\varphi$.
* **Policy Synthesis Problem**: given a plant $F$ and specification $\varphi$, find a policy $\pi$ such that $\Psi(F, \pi)\vDash\varphi$.

<h1>Framework</h1>

* The demonstration is a finite sample set $O_i:\{(x_1, U_1), (x_2, U_2), \ldots, (x_i, U_i)\}$ where $x_j\in X, U_j\subset \mathcal{U}$.
* A policy $\pi$ is `compatible` with $O_i$ iff for all $(x_j, U_j)\in O_i, \pi(x_j)\in U_j$.
* `Learner` gathers sample set $O$ in each iteration and learns a policy based on the sample set.
* `Demonstrator` D provides a set of feasible feedback $D(x)=U\subset \mathcal{U}$ for a given state $x$.
* `Verifier` asserts whether the intermediate policy $\pi$ satisfies the specification $\varphi$.

<h2>Algorithm</h2>
    
* **Initialization**
    Initial policy space $\Pi$ and an initial sample set $O_0=\emptyset$.

* **Iteration**<ol>
    <li> The learner chooses a policy $\pi_i\in\Pi$ that is compatible with $O_{i-1}$.</li>
    <li> The verifier either accepts $\pi_i$ or reject it with a counterexample trace $\sigma_i$.</li>
    <li> If there is a counterexample $\sigma_i$, then the demonstrator find $x_i$ where $\pi_i$ is not compatible with the demonstrator $\pi_i(x_i)\notin D(x_i)$.</li>
    <li> The learner updates $O_i: O_{i-1}\cup\{(x_i, D(x_i)\}$.
    </ol>
* **Terminate**
    If the framework terminates with success, then a policy satisfying $\varphi$ is obtained.
    
<h2> Demonstrator</h2>

* Let $int(G)$ be the set of interior of the goal state set $G$.
* The demonstrator's correctness is certified by a smooth Lyapunoav-like (value) function $V$ such that 
    * for all $x\in X$\$int(G), V(x)\geq 0$;
    
    * for all $V$ is radially unbounded;
    
    * for all $x\in X$\$int(G)$ with $u=\pi(x)$, it holds that $\nabla_x V\cdot F(x, u)\leq -\epsilon$
    > V is not required to be solved. Yet $\nabla V$ can be computed.
   
* Discretize the time step with $\delta$ and consider finite horizon MPC trajectory at time nodes $\delta, 2\delta, \ldots, N\delta$.
* Let $Q$ be the cost function for non-exit states and $H$ for the exit state.
* A suitable proof (certificate) $V$ for demonstrator's policy is the optimal value of the following problem.
\begin{align}
min_{u(0), \ldots, u(N\delta - \delta)}&& \sum^{N-1}_{j=0} Q(u(j\delta), x(j\delta)) + H(x(N\delta))\\
s.t.&& x(0)=x\\
&&x((j+1)\delta)=\hat{F}(x(j\delta), u(j\delta))\\
&&j=0,\ldots, N-1
\end{align}
* Initial state $x$ is determined beforehand.
* Cost functions $Q, H$ can be adjusted by trial and error until the demonstrator's control inputs work well from the initial state and the **cost decreases strictly along each of the resulting trajectories**.
* The gradient $\nabla V$ can be estimated using KKT conditions or using a numerical scheme that estimates the gradient by sampling around $x$.

<h2>Verifier</h2>

* To verify the reachability property of a nonlinear system is non-trivial. 

* Use falsifier that invert the demonstrator's optimization problem as a search heuristic for an initial state that likely to generate a counterexample to the property of reaching the goal G.
\begin{align}
max_{x(0)}&& \sum^{N-1}_{j=0} Q(u(j\delta), x(j\delta)) + H(x(N\delta))\\
s.t. &&x((j+1)\delta)=\hat{F}(x(j\delta), u(j\delta))\\
&&j=0,\ldots, N-1
\end{align}

* The policy $\pi$ and $u(j\delta)$ are known a priori before the falsification. 
* The initial state $x(0)$ can be found via random choice and gradient descent search.
* After a counterexample $\sigma$ is found, retrieve each state $x=\sigma(t)$ in time sequence. Treat $x$ as initial state and solve the optimal control $u^*=D(x)$ for it. If the cost decrement $\nabla V\cdot F(x, u)$ in this state in the counterexample is no smaller than $\nabla V\cdot F(x, u^*)$ of the optimal control up to some constant times, then return this state $x$ to demonstrator.

<h2>Learner</h2>

* Find a policy compatible with current the sample set $O$ in that $(\forall(x_j, U_j)\in O) \pi_\theta(x)\in U$.

* Use linear combination of basis functions to approximate the nonlinear policy $\pi_\theta(x):=\sum^K_{k=1} \theta_k v_k(x)$.

* Assume that each $U_j$ is a polyhedron $U_j:\{u|A_j u\leq b_j\}$

* Then $\pi_\theta(x_j)\in U_j$ can be replaced with $A_j(\sum^K_{k=1} \theta_k v_k(x_j))\leq b_j$ which is a polyhedron over $\theta$-space.

* If the polyhedron is too small, then the learning fails. Otherwise, find the center of the maximum volumn ellipsoid (MVE) in the polyhedron and derive the policy.




<h1>Case Study</h1>

<h3> A car with two axles</h3>

* The goal is to follow a reference curve by controlling the lateral deviation from the midpoint of the reference at a constant speed.

* Relative velocity is used. The cost functions are given. 

* The MPC proposed in this paper can solve the control problem for many random initial states whereas a LQR controller with the same cost function fails (not reaching the goal states).

* The method in this paper converges faster when the policy is linear as opposed to being affine. 

* The method works well in Webots simulator. The learnt controller is robust to compensate the model mismatches.

<h3> Caltech Ducted Fan</h3>

* Need some manipulation to acquire affine dynamics.

* Results suggest that the frameworkd can efficiently yield reliable solution to reachability problems.

* In the same scenario, linear regression is also tried. No matter how large the training data set is, the falsifier is still able to find counterexample for the learnt policy.